In [24]:
import numpy as np
import os
import matmul as mm
import math_util as mu
import scipy.io as io
from amm_methods import *
import socket # Obtain the current host name, which can be used to select different data directories and result saving directories
import math

In [25]:
# method = METHOD_MITHRAL
# method = METHOD_PQ
# method = METHOD_EXACT
method = METHOD_SCALAR_QUANTIZE
user_method = "METHOD_USER_SCALAR_QUANTIZE"
# user_method = "METHOD_USER_QUANTIZE"


In [26]:
feedback_bits = 256
ncodebooks=64
ncentroids=256
train_sam_num = 3000 # 训练集样本数
qbits = 8
prop_A = 0
prop_B = 0

In [27]:
host_name = socket.gethostname()
if host_name == 'DESKTOP-PLRL7TK':
    dir_train = 'E:\\hdr\\研一\\华为-深度学习\\intermediate\\intermediate8dbfc1'
    dir_result = ''
elif host_name == 'DESKTOP-6FOH47P':
    dir_train = 'F:\\Projects\\python\\PQ\\intermediate8dbfc1'
    dir_result = 'F:\\Projects\\python\\PQ\\res'
    data_to_fcpath_train= 'data_to_fc_e39_7999.npy'
    featurepath_train= 'feature_e39_7999.npy'
    data_to_fcpath_test = 'data_to_fc_e39_7999.npy'
    featurepath_test = 'feature_e39_7999.npy'
elif host_name == 'jm-System-Product-Name':
    dir_train = '/data/hdr/transformer_data/joined'
    dir_result = '/data/hdr/pq/res'
    data_to_fcpath_train= 'data_to_fc_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    y_train = 'y_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    featurepath_train= 'feature_train_f%i_sam%i.npy' % (feedback_bits, train_sam_num)
    data_to_fcpath_test = 'data_to_fc_test.npy'
    featurepath_test = 'feature_test.npy'
    y_test = 'y_test.npy'
else:
    raise NameError("You are running the script in a new computer, please define dir_train")


weightpath = 'encoder_fcw.npy'
biaspath = 'encoder_fcb.npy'



In [28]:
# from turtle import right


def saturation_sort(A, proportion = 0.3, axis = 0): #饱和排序：proportion:饱和比例（一般为30%）
    axis_quantity = A.shape[axis] # axis = 0返回行数，1返回列数
    
    mins_ori = A.min(axis=axis, keepdims=True)
    # A_offset = A - offsets
    ranges_ori = (A - mins_ori).max(axis=axis, keepdims=True) + 1e-20
    if proportion == 0:
        return mins_ori, ranges_ori
    elif proportion > 0 and proportion <= 1:
        ix_threshold = math.floor(proportion*axis_quantity) # 前proportion%的最后一位
        Ato0 = A - ranges_ori/2. - mins_ori # 使A的axis维的最大最小值关于0对称
        Ato0abs = np.absolute(Ato0)
        Ato0abssort = np.sort(Ato0abs, axis=axis)
        if axis == 0:
            threshold = Ato0abssort[np.ix_([axis_quantity - ix_threshold], range(Ato0abssort.shape[1]))]
        elif axis == 1:
            threshold = Ato0abssort[np.ix_(range(Ato0abssort.shape[0]), [axis_quantity - ix_threshold])]
        mins_new = ranges_ori/2. + mins_ori - threshold
        ranges_new = 2 * threshold
        # print('ranges_ori')
        # print(ranges_ori)
        # print('mins_ori')
        # print(mins_ori)
        # print('Ato0')
        # print(Ato0)
        # print('Ato0abs')
        # print(Ato0abs)
        # print('Ato0abssort')
        # print(Ato0abssort)
        # print('threshold')
        # print(threshold)
        return mins_new, ranges_new


In [29]:
# zx = np.random.randint(1,100,(11,3))
# print(zx)
# mins_new, ranges_new = saturation_sort(zx)
# print(mins_new, ranges_new)

In [30]:
def _scalar_quantize(A, proportion = 0.3, axis=0, signed=False, nbits=8):
    unsigned_maxval = float(1 << int(nbits)) - 1

    # # TODO rm
    # # return np.zeros((A.shape[0], 1)), np.ones((A.shape[0], 1)), A
    # # offsets = np.zeros((A.shape[0], 1))
    # offsets = A.min(axis=1, keepdims=True)
    # # scales = maxval / np.ones((A.shape[0], 1))
    # scales = maxval / A.max(axis=1, keepdims=True)
    # Aq = (A - offsets) * scales
    # return offsets, scales, Aq
    mins, ranges = saturation_sort(A, proportion = proportion, axis = axis)
    # maxval = float(1 << int(nbits)) - 1
    # mins = A.min(axis=axis, keepdims=True)
    # A_offset = A - offsets
    # ranges = (A - mins).max(axis=axis, keepdims=True) + 1e-20
    # print("ranges:")
    # print(ranges)
    scales = unsigned_maxval / ranges
    # Aq = (A_offset * (maxval / scales)).astype(np.int)
    # Aq = (A_offset * scales).astype(np.int)

    if signed:
        # sign_offset = 1 << (nbits - 1)  # 8 bits -> 128
        # A_offset -= sign_offset
        offsets = mins + (ranges * (128. / 255))
        minval = -(1 << (nbits - 1))
        maxval = -minval - 1
    else:
        offsets = mins
        minval = 0
        maxval = (1 << nbits) - 1

    Aq = (A - offsets) * scales
    # print("min, max A:", Aq.min(), Aq.max())  # looks good
    Aq = np.clip(Aq, minval, maxval).astype(int)

    return offsets, scales, Aq, minval, maxval

In [31]:
def test_user_quantize(A, B, offsets, scales, minval, maxval, axis=0, signed=False, nbits=8):
    Aq = (A - offsets) * scales
    Aq = np.clip(Aq, minval, maxval).astype(int) # 量化后的值（整数
    Aqreal = Aq / scales + offsets # 量化后对应的实际值
    print("A:")
    print(A)
    print("Aq:")
    print(Aq)
    print("Aqreal")
    print(Aqreal)
    C = np.matmul(Aqreal, B)

    return C

In [ ]:
def test_user_scalar_quantize(A, B, A_offsets, A_scales, A_minval, A_maxval, B_offsets, B_scales, B_minval, B_maxval):
    Aq = (A - A_offsets) * A_scales
    Aq = np.clip(Aq, A_minval, A_maxval).astype(int) # 量化后的值（整数
    Aqreal = Aq / A_scales + A_offsets # 量化后对应的实际值

    Bq = (B - B_offsets) * B_scales
    Bq = np.clip(Bq, B_minval, B_maxval).astype(int) # 量化后的值（整数
    Bqreal = Bq / B_scales + B_offsets # 量化后对应的实际值

    # print("A:")
    # print(A)
    # print("Aq:")
    # print(Aq)
    # print("Aqreal")
    # print(Aqreal)
    C = np.matmul(Aqreal, Bqreal)

    return C

In [1]:
# ran_mat = np.rint(10*np.random.random(size=(5,5)))
# print(ran_mat)

In [33]:
# offsets, scales, Aq, minval, maxval = _scalar_quantize(ran_mat, proportion=prop, axis=0, signed=False, nbits=8)

In [34]:
# print(offsets)
# print(scales)
# print(Aq)
# print(minval)
# print(maxval)

[[0. 1. 0. 2. 1.]]
[[51.         36.42857143 42.5        36.42857143 28.33333333]]
[[  0 255 170  36   0]
 [255  72 255 255  28]
 [204 109  85 109 255]
 [ 51   0   0   0 141]
 [102 109 127 255  28]]
0
255


In [35]:
data_to_fc_train = np.load(os.path.join(dir_train, data_to_fcpath_train))
print(data_to_fc_train.shape)
weight = np.load(os.path.join(dir_train, weightpath))
print(weight.shape)
data_to_fc_test = np.load(os.path.join(dir_train, data_to_fcpath_test))
print(data_to_fc_test.shape)
bias = np.load(dir_train+'/'+biaspath)

(96000, 2048)
(2048, 256)
(32000, 2048)


In [36]:
train_offsets, train_scales, train_Aq, train_minval, train_maxval = _scalar_quantize(data_to_fc_train, proportion=prop, axis=0, signed=False, nbits=qbits)

In [37]:
if user_method == "METHOD_USER_QUANTIZE":
    y_out_matmul = test_user_quantize(data_to_fc_test, weight, train_offsets, train_scales, train_minval, train_maxval)
    y_out_last = y_out_matmul + bias.T # MADDNESS替换后当前层输出，即+bias并不需要激活函数后的结果

A:
[[ 0.001161   -0.0003292   0.00079327 ...  0.02352978  0.02011615
   0.01869451]
 [-0.00108941 -0.0003551  -0.00034218 ...  0.01012411  0.00046477
   0.00978941]
 [ 0.00097858  0.00035693  0.00280184 ... -0.01709364  0.01152093
   0.00830318]
 ...
 [ 0.0044448  -0.00065186  0.00043888 ... -0.0145436  -0.00225845
   0.00354694]
 [ 0.00094269 -0.0011471   0.00078288 ...  0.00279342  0.00330969
  -0.01324401]
 [ 0.00324066  0.00071579  0.00377456 ... -0.0036882   0.00070076
  -0.00470257]]
Aq:
[[163 130 181 ... 255 255 255]
 [ 20 126  62 ... 135 102 255]
 [151 235 255 ...   0 251 255]
 ...
 [255  81 144 ...   0  65 223]
 [149   5 179 ...  66 140  57]
 [255 255 255 ...   6 105 142]]
Aqreal
[[ 0.00115585 -0.000333    0.00079299 ...  0.02292155  0.01175397
   0.00669053]
 [-0.00109537 -0.00035916 -0.00034607 ...  0.01007552  0.00046017
   0.00669053]
 [ 0.00096694  0.00035359  0.00150132 ... -0.00437626  0.01145871
   0.00669053]
 ...
 [ 0.00260419 -0.00065341  0.00043883 ... -0.00437626 

In [38]:
if user_method == "METHOD_USER_QUANTIZE":
    print(y_out_last)
    print(y_out_last.dtype)
    print(y_out_last.shape)
    np.save(dir_result+'/'+'UserQuantize'+'fc1_prop%0.3f_fb%i_qb%i.npy' % (prop, feedback_bits, qbits), y_out_last.astype(np.float32))


[[-0.0632925   0.11510547 -0.38953638 ... -0.00586694 -0.0227966
   0.07346666]
 [-0.11025896  0.01035206 -0.18747466 ... -0.22178045 -0.02247104
  -0.01913035]
 [ 0.09450215  0.10371651 -0.19693512 ... -0.1460753   0.02490801
   0.15774144]
 ...
 [ 0.1046813   0.06164711 -0.35373129 ... -0.03216408 -0.10479045
  -0.08956613]
 [ 0.1305362   0.0284576  -0.22534953 ...  0.03662261 -0.01087786
   0.03341703]
 [ 0.00086727  0.11816691 -0.30380601 ... -0.09984724 -0.1072027
  -0.00366244]]
float64
(32000, 256)


In [2]:
# print(train_offsets.shape)
# print(train_scales.shape)
# print(train_Aq.shape)

In [3]:
# print(train_offsets)

In [ ]:
if user_method == "METHOD_USER_SCALAR_QUANTIZE":
    weight_offsets, weight_scales, weight_Aq, weight_minval, weight_maxval = _scalar_quantize(weight, proportion=prop, axis=1, signed=False, nbits=qbits)